In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('./train.csv', dtype=np.float32)

In [3]:
labels = df.label.values
features = df.loc[:,df.columns != 'label'].values/255

In [4]:
features_train, features_val, labels_train, labels_val = train_test_split(features,labels,train_size=0.95,random_state=32)

C:\Users\prate\AppData\Local\conda\conda\envs\mlenv\lib\site-packages\sklearn\model_selection\_split.py:2069: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
trainset = TensorDataset(torch.from_numpy(features_train), torch.from_numpy(labels_train))
valset  = TensorDataset(torch.from_numpy(features_val),torch.from_numpy(labels_val))

In [6]:
train_loader = DataLoader(trainset, batch_size=5, shuffle=True)
val_loader = DataLoader(valset, batch_size=5, shuffle=False)

In [7]:
#Model - Multi layer NN 
class MultiLayerPerceptron(nn.Module):
    def __init__(self, in_features, out_features):
        super(MultiLayerPerceptron, self).__init__()
        self.l1 = nn.Linear(in_features=in_features,out_features=150)
        self.l2 = nn.Linear(in_features=150,out_features=150)
        self.l2 = nn.Linear(in_features=150,out_features=150)
        self.l3 = nn.Linear(in_features=150,out_features=out_features)
        self.relu = nn.ReLU(inplace=True)
        nn.init.kaiming_normal_(self.l1.weight)
        nn.init.kaiming_normal_(self.l2.weight)
        nn.init.kaiming_normal_(self.l3.weight)
    def forward(self, x):
        x = self.l1(x)
        x = self.l2(self.relu(x))
        x = self.l3(self.relu(x))
        return x

net = MultiLayerPerceptron(784,10)

In [ ]:
# crossentropy loss
criteria = nn.CrossEntropyLoss()
#optimizer
optimizer = optim.SGD(net.parameters(),lr=0.01)
epochs = 5

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5000, gamma=0.1)

In [35]:
net.train()
train_loss = []
for e in range(epochs):
    for i, (img, label) in enumerate(train_loader):
        scheduler.step()

        #wrapper
        img = Variable(img.view(-1,28*28))
        label = Variable(label).type(torch.LongTensor)
        optimizer.zero_grad()

        out = net(img)
        loss = criteria(out, label)
        train_loss.append(loss.item())

        loss.backward()
        optimizer.step()
        if i %1000 ==0:
            accuracy,correct = 0,0
            net.eval()
            for val_img, val_label in val_loader:
                
                #wrapper
                val_img = Variable(val_img.view(-1,28*28))
                
                outs = net(val_img)
                
                predictions = torch.max(outs.data,1)[1]
                correct += torch.sum(predictions==val_label.type(torch.LongTensor))

            accuracy = 1.*correct.item()/float(len(valset))
            print('Epoch {} iteration {} Valaccuracy {} loss {}'.format(e, i, accuracy, loss.item()))
            net.train()

Epoch 0 iteration 0 Valaccuracy 0.9714285714285714 loss 3.299712989246473e-05
Epoch 0 iteration 1000 Valaccuracy 0.9709523809523809 loss 0.0005260467296466231
Epoch 0 iteration 2000 Valaccuracy 0.97 loss 0.00780906667932868
Epoch 0 iteration 3000 Valaccuracy 0.97 loss 0.00041790009709075093
Epoch 0 iteration 4000 Valaccuracy 0.9719047619047619 loss 0.0005401611560955644
Epoch 0 iteration 5000 Valaccuracy 0.9714285714285714 loss 0.0005321502685546875
Epoch 0 iteration 6000 Valaccuracy 0.9714285714285714 loss 0.0001697540283203125
Epoch 0 iteration 7000 Valaccuracy 0.9709523809523809 loss 0.00013427734666038305
Epoch 1 iteration 0 Valaccuracy 0.9704761904761905 loss 0.00016345977201126516
Epoch 1 iteration 1000 Valaccuracy 0.9709523809523809 loss 1.659393274167087e-05
Epoch 1 iteration 2000 Valaccuracy 0.9704761904761905 loss 0.015848446637392044
Epoch 1 iteration 3000 Valaccuracy 0.9704761904761905 loss 1.5258789289873675e-06
Epoch 1 iteration 4000 Valaccuracy 0.9704761904761905 loss 0.

In [12]:
testdf = pd.read_csv('./test.csv', dtype=np.float32)
test_features = torch.from_numpy(testdf.values)
out = net(test_features)

In [47]:
test_predictions = torch.max(out,1)[1]

In [46]:
#save to csv
testdf = pd.DataFrame({'ImageId': testdf.index.values+1,'Label':test_predictions.numpy()})
testdf.to_csv('./pred.csv', index=False)